In [5]:
# import requests

# # Define the URL of your FastAPI server
# url = "http://localhost:8000"  # Update with your server's URL

# # Define the data to send as JSON
# data = {
#     "text1": "Input Text 1",
#     "text2": "Input Text 2"
# }

# # Send a POST request with JSON data
# response = requests.post(url, json=data)

# # Check the response
# if response.status_code == 200:
#     result = response.json()
#     similarity = result["similarity"]
#     print(f"Cosine Similarity: {similarity}")
# else:
#     print(f"Request failed with status code: {response.status_code}")

In [3]:
import requests

# Define the URL of your FastAPI server
url = "http://localhost:8000/add_person"  # Update with your server's URL

# Define the data to send as JSON
person_data = {
    "first_name": "enas",
    "last_name": "khwaileh",
    "age": 35
}

# Send a POST request with JSON data to add a person
response = requests.post(url, json=person_data)

# Check the response
if response.status_code == 200:
    result = response.json()
    print(result["message"])
else:
    print(f"Request failed with status code: {response.status_code}")


Person added successfully


In [2]:
import os
os.listdir("uploads")

['test.csv', 'test2.csv', 'test3.csv']

In [13]:
import sentence_transformers
from sentence_transformers import SentenceTransformer, util
import numpy as np
import pandas as pd


db = np.array(['Data Scientist',' Machine Learning Engineer','Data Analyst', 'Software Developer','Front End Developer','Back End Developer','Mathematician','Physicist'])
model = SentenceTransformer("all-mpnet-base-v2")
encodings = model.encode(db)
k=5
query = 'ML Engineer'
query_emb = model.encode(query)
cos_sims = util.cos_sim(query_emb, encodings).numpy()[0]
top_k_indices = np.argsort(-cos_sims)[:k]

top_k_results = list(zip(db[top_k_indices],cos_sims[top_k_indices]))
top_k_results = pd.DataFrame(top_k_results, columns=['TableName','SimilarityScore']).to_dict()

{'TableName': {0: ' Machine Learning Engineer',
  1: 'Data Scientist',
  2: 'Data Analyst',
  3: 'Software Developer',
  4: 'Physicist'},
 'SimilarityScore': {0: 0.8346510529518127,
  1: 0.6666718125343323,
  2: 0.6132876873016357,
  3: 0.563723087310791,
  4: 0.5109235644340515}}

In [1]:
import os
from os.path import isfile, join
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import torch
from prettytable import PrettyTable


c:\Users\leandros\Documents\GitHub\mira_gui\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
datasetpath =  f"semantic_matching/data/GroundTruth_Data/"
embeddingspath = f"semantic_matching/data/newdataembedding/"

files = [f for f in os.listdir(datasetpath) if (isfile(join(datasetpath, f)) and f.endswith('.csv'))]
embeddingsFiles = [f for f in os.listdir(embeddingspath) if (isfile(join(embeddingspath, f)) and f.endswith('.pt'))]

embeddings = torch.empty(0, device = 'cpu')
for embeddingsFile in embeddingsFiles:
    embeddingsOffile = torch.load(embeddingspath+ embeddingsFile, map_location=torch.device('cpu'))
    embeddings = torch.concat([embeddings, embeddingsOffile])

In [15]:
filesContent = {}
content = []

for file in files:
    try:
        df = pd.read_csv( datasetpath + file, encoding = "ISO-8859-1", on_bad_lines = "skip")
        filesContent[file] = df.columns.tolist()
        content += df.columns.tolist()
        for row in df.values:
            filesContent[file] += list(map(str,row))
            content += list(map(str,row))
    except Exception as e:
        print(f"Error in reading a csv file: file name: {file}, Error message: {e}")

Error in reading a csv file: file name: Airbnb_Open_Data.csv, Error message: Error tokenizing data. C error: EOF inside string starting at row 36


In [5]:
os.listdir(datasetpath)

['GroundTruth_Data']

In [19]:

model = SentenceTransformer("all-mpnet-base-v2")

query_embedding1 = model.encode('Computer Scientist')
query_embedding2 = model.encode('parental level of education')
cos_sims = util.cos_sim(query_embedding1, query_embedding2).numpy()[0]
cos_sims

array([0.18616231], dtype=float32)

In [ ]:
import os
from os.path import isfile, join
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import torch
from prettytable import PrettyTable

def readFiles():

	datasetpath =  f"../data/GroundTruth_Data/"
	embeddingspath = f"../data/newdataembedding/"

	files = [f for f in os.listdir(datasetpath) if (isfile(join(datasetpath, f)) and f.endswith('.csv'))]
	embeddingsFiles = [f for f in os.listdir(embeddingspath) if (isfile(join(embeddingspath, f)) and f.endswith('.pt'))]
	
	embeddings = torch.empty(0, device = 'cpu')
	for embeddingsFile in embeddingsFiles:
		embeddingsOffile = torch.load(embeddingspath+ embeddingsFile)
		embeddings = torch.concat([embeddings, embeddingsOffile])

	
	filesContent = {}
	content = []
	
	for file in files:
		try:
			df = pd.read_csv( datasetpath + file, encoding = "ISO-8859-1", on_bad_lines = "skip")
			filesContent[file] = df.columns.tolist()
			content += df.columns.tolist()
			for row in df.values:
				filesContent[file] += list(map(str,row))
				content += list(map(str,row))
		except Exception as e:
			print(f"Error in reading a csv file: file name: {file}, Error message: {e}")
		
	return filesContent, content, embeddings

def getMeXResults(X,top_results, filesContent, content):
	XResults = []
	i = 0
	cellsFileCombi = []
	for score, idx in zip(top_results[0], top_results[1]):
		if float("{:.4f}".format(score)) >= 0.3:
			for filename, val in filesContent.items():
				if content[idx] in val:
					if (filename, content[idx]) not in cellsFileCombi:
						cellsFileCombi.append((filename, content[idx]))
						XResults.append((filename, content[idx], score))
						i += 1
						if i == X:
							return XResults
	return XResults

def results(query, filesContent, content, embeddings):   
	
	symmetric_embedder = SentenceTransformer('all-mpnet-base-v2')
	results = {}
	
	query_embedding = symmetric_embedder.encode(query, convert_to_tensor = True)
	top_k = min(10, len(embeddings))
	cos_scores = util.cos_sim(query_embedding, embeddings)[0]
	top_results = torch.topk(cos_scores, len(embeddings), sorted = True)
	
	results = getMeXResults(top_k, top_results, filesContent, content)
	return results

def matching():
	filesContent, content, embeddings = readFiles()
	while True:
		query = "Rock Music"
		result = results(query, filesContent, content, embeddings)
		print("============================")
		print(f"your query is: {query}")
		
		table = [['Filename', 'Cell Value', 'Score']]
		for i in result:
			table.append([i[0], i[1], "{:.4f}".format(i[2])])
		tab = PrettyTable(table[0])
		tab.add_rows(table[1:])
		print(tab)
		
if __name__ == "__main__":
	matching()


# Embeddings generation

In [148]:
import os
import torch
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util

In [ ]:
model = SentenceTransformer("all-mpnet-base-v2")

In [152]:

def create_embeddings(table_name, base_path):

    if os.path.exists(base_path + table_name.replace('.csv', '.pt') ):
        return 'Embeddings file already exist'

    df = pd.read_csv(base_path + table_name)

    col_names = df.columns.tolist()
    df_columns = pd.DataFrame([col_names], columns=col_names)

    df = pd.concat([df_columns,df], ignore_index=True)
    df = df.map(lambda cell_value: str(cell_value))
    cell_values_flat = df.values.flatten()

    # Create embeddings for all cell values
    embeddings_list_flat = model.encode(cell_values_flat)

    # Save embeddings to pt file
    torch.save(embeddings_list_flat, base_path + table_name.replace('.csv', '.pt') )

    return 'Embeddings created successfully'

In [158]:
table_name = 'shopping_trends.csv'
base_path = 'semantic_matching/data/'

create_embeddings(table_name, base_path)

'Embeddings created successfully'

In [162]:
def create_data(table_name):

    print(f'Processing table: {table_name}')

    # Read csv data
    # base_path = 'semantic_matching/data/'
    df = pd.read_csv(table_name)

    # Add column names as first row
    col_names = df.columns.tolist()
    df_columns = pd.DataFrame([col_names], columns=col_names)
    df = pd.concat([df_columns,df], ignore_index=True)
    df = df.map(lambda cell_value: str(cell_value))
    cell_values_flat = df.values.flatten()

    # Create embeddings for all cell values
    # embeddings_list_flat = model.encode(cell_values_flat)
    embeddings_path = table_name.replace('.csv', '.pt')
    assert os.path.exists(embeddings_path), "Embeddings path doesn't exist"
    embeddings_list_flat = torch.load(embeddings_path)
    
    # cell_numbers_flat = list(range(len(cell_values_flat)))

    col_names_flat = col_names * int((len(cell_values_flat)/len(col_names)))

    # Final DataFrame
    name_and_embs = pd.DataFrame(list(zip([table_name.split('/')[-1]]*len(embeddings_list_flat), cell_values_flat , col_names_flat, embeddings_list_flat)), columns=['TableName','CellValue','CellValue_Column','Embeddings'])

    return name_and_embs


In [126]:
def get_top_k(query, input_df, k):

    query_emb = model.encode(query)
    cos_sims = util.cos_sim(query_emb, input_df['Embeddings']).numpy()[0]
    input_df['SimilaritiyScores'] = cos_sims
    input_df =  input_df.drop_duplicates(subset='CellValue').reset_index(drop=True)
    input_df =  input_df.sort_values('SimilaritiyScores', ascending=False)

    return input_df.iloc[:k]

In [140]:
def batch_semantic_matching(query, table_names, k):

    total_results_df = pd.DataFrame([],columns=['TableName','CellValue','CellValue_Column','Embeddings'])

    for table_name in table_names:

        # Calculate cosine similarities 
        res_df = create_data(table_name)
        total_results_df = pd.concat([total_results_df, res_df], ignore_index=True)

    # Get top-k similar results
    top_k_df = get_top_k(query, total_results_df, k).reset_index(drop=True)

    return top_k_df


In [160]:
# Get all csv files:
base_path = 'semantic_matching/data/'
csv_files = [base_path+file for file in os.listdir(base_path) if file.endswith('.csv')]
csv_files

['semantic_matching/data/jobs.csv',
 'semantic_matching/data/shopping_trends.csv']

In [161]:
csv_files[0].split('/')[-1]

'jobs.csv'

In [163]:
# table_names = ['jobs.csv', 'shopping_trends.csv']

batch_semantic_matching('Fashion Data Analyst', csv_files, 5)

Processing table: semantic_matching/data/jobs.csv
Processing table: semantic_matching/data/shopping_trends.csv


,TableName,CellValue,CellValue_Column,Embeddings,SimilaritiyScores
0,shopping_trends.csv,Clothing,Category,"[0.02621154, 0.04037507, -0.0047923974, -0.022...",0.569970
1,jobs.csv,Data Analyst,Job Name,"[-0.015574576, 0.04354819, -0.07052371, -0.018...",0.544447
2,jobs.csv,Data Scientist,Job Name,"[-0.0039955433, 0.064525425, -0.065324955, -0....",0.461979
3,shopping_trends.csv,Blouse,Item Purchased,"[-0.007990204, 0.018430114, -0.017955607, -0.0...",0.420859
4,shopping_trends.csv,Dress,Item Purchased,"[-0.019607369, 0.064712085, -0.0047004707, 0.0...",0.410074


In [115]:
# res_df = create_data('jobs.csv')

Processing table: jobs.csv


In [117]:
# query_emb = model.encode('Analyst')
# cos_sims = util.cos_sim(query_emb, res_df['Embeddings']).numpy()[0]
# res_df['SimilaritiyScores'] = cos_sims
# res_df =  res_df.drop_duplicates(subset='CellValue').reset_index(drop=True)
# # res_df['SimilarityScore_Rank'] = res_df['SimilaritiyScores'].rank(method='dense', ascending=False)
# # res_df = res_df.sort_values('SimilaritiyScores', ascending=False)
# # top_k_indices = np.argsort(-cos_sims)[:k]

# # res = res_df.iloc[top_k_indices].reset_index(drop=True)
# # res['SimilaritiyScores'] = cos_sims[top_k_indices]

In [ ]:
# res_df.sort_values('SimilaritiyScores', ascending=False).iloc[:5]

In [ ]:
# res_df = create_data(table_names[0])


In [65]:
# top_k_df = get_top_k('Job', res_df, 5)

In [46]:
# embeddings_data = test_table.map(lambda cell_value: model.encode(str(cell_value)))
# embeddings_list_flat= embeddings_data.values.flatten()
# cell_values_flat = test_table.values.flatten()
# cell_numbers_flat = list(range(len(cell_values_flat)))

In [ ]:
# col_names_flat = col_names * int((len(cell_values_flat)/len(col_names)))

In [ ]:
# name_and_embs = pd.DataFrame(list(zip([test_table_name]*len(embeddings_list_flat), cell_values_flat , col_names_flat, embeddings_list_flat)), columns=['TableName','CellValue','CellValue_Column','Embeddings'])
# name_and_embs

In [ ]:
# k=5
# query_emb = model.encode('Job')
# cos_sims = util.cos_sim(query_emb, name_and_embs['Embeddings']).numpy()[0]
# top_k_indices = np.argsort(-cos_sims)[:k]

# res = name_and_embs.iloc[top_k_indices].reset_index(drop=True)
# res['SimilaritiyScores'] = cos_sims[top_k_indices]
# res

# Qdrant

In [21]:
import qdrant_client
from qdrant_client import QdrantClient

client = QdrantClient("localhost", port=6333)

In [23]:
from qdrant_client.http.models import Distance, VectorParams

client.create_collection(
    collection_name="test_collection",
    vectors_config=VectorParams(size=768, distance=Distance.COSINE),
)

True